In [4]:
import pickle  
import networkx as nx
import random
import numpy as np
import copy
import pandas as pd
import itertools
import math
from tqdm import tqdm
from scipy import integrate 
from collections import defaultdict
from find_motifs import * #佳伟寻找模体的库
from Motif_structures import * #轨道度结构

In [6]:
moti = [M1, M2, M3, M4, M5, M6, M7, M8, M9, M10, M11, M12]
moti_name = ['M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'M10', 'M11', 'M12']
motif_index = []
zip1 = zip(moti, moti_name)
for i, j in zip1:
    motif_index.append((i, j))
    
noti = [N1, N2, N3, N4, N5, N6, N7, N8, N9, N10, N11, N12, N13, N14, N15]
noti_name = ['N1', 'N2', 'N3', 'N4', 'N5', 'N6', 'N7', 'N8', 'N9', 'N10', 'N11', 'N12', 'N13', 'N14', 'N15']
notif_index = []
zip2 = zip(noti, noti_name)
for i, j in zip2:
    notif_index.append((i, j))

In [7]:
infile = open('OLP_updated.pickle','rb')  
df = pickle.load(infile)
infile.close()  
# read edge lists for all networks
df_edgelists = df['edges_id']                               
all_name = list(df['network_name'])

for i in tqdm(range(len(df))):
    file_name = './features/n_'+str(i)+'_features.csv'
    ###################构建网络，并获取其属性##########################
    edges_orig = df_edgelists.iloc[i] # a numpy array of edge list for original graph
    file = df['network_name'][i]
    num_edges = df['number_edges'][i] #下面是第i个网络的一些基本属性
    num_nodes = df['number_nodes'][i]
    ave_degree = df['ave_degree'][i]
    domain = df['networkDomain'][i] 
    g1 = nx.Graph()
    g1.add_edges_from(edges_orig)
    
    edges = list(g1.edges())
    nodes = list(g1.nodes())
    node_num = len(list(g1.nodes()))
    remove_edge_num = int(len(edges)*0.2)#训练集边的数量 
    all_edges = list(g1.edges())
    edge_all = list(g1.edges())
    
    
    
    ####################train network#######################
    train_g = copy.deepcopy(g1)
    remove_edges = random.sample(edge_all, remove_edge_num)#20%的missing links
    train_g.remove_edges_from(remove_edges)
    
    #positive sample
    positive_samples = remove_edges
    
        ########################  build nonexistent samples ####################     
    negitive_samples = []                             
    while len(negitive_samples) < len(positive_samples):
        u = random.choice(nodes)# random a node
        v = random.choice(nodes)
        if u != v and (u,v) not in all_edges and (v,u) not in all_edges and (u,v) not in negitive_samples and (v,u) not in negitive_samples:
            negitive_samples.append((u,v)) 
    ###################### end ################################
    
    X = defaultdict(list)
    X['edge'] = positive_samples + negitive_samples
    X['label'] = [1]*len(positive_samples) + [0]*len(negitive_samples)
    
    all_samples = positive_samples + negitive_samples
    
    ###########################15 node orbit degree features###################
    individual_nodes = sorted(set(element for tup in all_samples for element in tup))
    node_orbit_degree_dict = {} #key：node，value：orbit degree list   
    for node in individual_nodes: 
        node_orbit_degree = []
        for n in noti:
            node_orbit_degree.append(node_orbit_motif_degree(train_g,n,node,1))
        node_orbit_degree_dict[node] = node_orbit_degree
    for j in range(len(noti_name)):
        res = []
        for one_node_pair in  all_samples:
            res.append(node_orbit_degree_dict[one_node_pair[0]][j]*node_orbit_degree_dict[one_node_pair[1]][j])
        X[noti_name[j]]= res
        
    
    ###########################12 edge orbit degree features###################
    for m in motif_index: 
        res = []
        for one in all_samples:
            a = edge_orbit_motif_degree(train_g,m[0],one,(1,2))
            res.append(a)
        X[m[1]]= res
    df_features = pd.DataFrame(X)
    
    df_features.to_csv(file_name,index = False)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  5.14it/s]
